In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_newyork.csv', delimiter=',')

In [3]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_newyork_max_recall5.csv', delimiter=',')

In [4]:
embedding_model = Doc2Vec.load('../pre_trained_models/doc2vec.bin')

In [5]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [6]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [7]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_full_text         141 non-null    object
 1   article_key_match         148 non-null    object
 2   article_keywords          143 non-null    object
 3   article_main_passage      148 non-null    object
 4   article_meta_description  132 non-null    object
 5   article_meta_keywords     58 non-null     object
 6   article_summary           141 non-null    object
 7   article_tags              54 non-null     object
 8   article_title             148 non-null    object
 9   article_url               148 non-null    object
dtypes: object(10)
memory usage: 11.7+ KB


In [8]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14800 entries, 0 to 14799
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              14800 non-null  object
 1   table_page_id            14800 non-null  object
 2   table_page_title         14800 non-null  object
 3   table_page_main_passage  14151 non-null  object
 4   table_page_keywords      10971 non-null  object
dtypes: object(5)
memory usage: 578.2+ KB


In [9]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_key_match']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])

        #embedding and model variables
        article_title1 = []
        article_title2 = []
        title_table1 = []
        title_table2 = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_key_match']]
        
        vector_words1 = tknzr.tokenize(article_title_text)
        word_embedding1 = embedding_model.infer_vector(vector_words1)
        article_title1.append(word_embedding1)
        
        vector_words2 = tknzr.tokenize(article_meta_description_text)
        word_embedding2 = embedding_model.infer_vector(vector_words2)
        article_title2.append(word_embedding2)

        #creating embedding 
        for i, row in return_index.iterrows():
            
            vector_words3 = tknzr.tokenize(str(row['table_page_title']))
            word_embedding3 = embedding_model.infer_vector(vector_words3)
            title_table1.append(word_embedding3)
            
            vector_words4 = tknzr.tokenize(str(row['table_page_main_passage']))
            word_embedding4 = embedding_model.infer_vector(vector_words4)
            title_table2.append(word_embedding4)
        
        distance_vector1 = pairwise_distances(article_title1, title_table1, metric='cosine')
        distance_vector2 = pairwise_distances(article_title2, title_table2, metric='cosine')
        final_vector = (np.array(distance_vector1) + np.array(distance_vector2)) / 2.0
        
        #creating the final dataframe
        for i in range(0,len(final_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],final_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

148it [09:16,  3.76s/it]
148it [09:10,  3.72s/it]
148it [08:35,  3.48s/it]
148it [08:01,  3.25s/it]
148it [06:43,  2.73s/it]


['Acc@1', '0.2162']
['Acc@5', '0.4527']
['Acc@10', '0.5676']
['Acc@20', '0.7432']
['Acc@50', '0.8986']
MRR: 0.3324
[0, 0.16666666666666666, 0.3333333333333333, 1.0, 0.5, 0.3333333333333333, 0, 0.125, 0.07142857142857142, 0.125, 0.3333333333333333, 0.045454545454545456, 0.5, 1.0, 1.0, 1.0, 0, 0.3333333333333333, 0, 1.0, 0.25, 0.09090909090909091, 0.5, 0.07692307692307693, 0.14285714285714285, 0.0625, 0.047619047619047616, 0.05, 0, 0.14285714285714285, 1.0, 0.25, 1.0, 0.022222222222222223, 0.5, 0.14285714285714285, 0.047619047619047616, 0.5, 0.02564102564102564, 0, 0.25, 0.25, 0.023809523809523808, 0.3333333333333333, 0.07142857142857142, 0, 1.0, 0.045454545454545456, 0.029411764705882353, 1.0, 0.3333333333333333, 0.125, 0.07142857142857142, 1.0, 0.08333333333333333, 0.02857142857142857, 0.045454545454545456, 1.0, 0.030303030303030304, 1.0, 0.02702702702702703, 0.07142857142857142, 0.08333333333333333, 0.07142857142857142, 1.0, 1.0, 0.045454545454545456, 1.0, 0.0625, 1.0, 0.05, 0.076923